In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook, Workbook
from tkinter import Tk, filedialog, messagebox
import tkinter as tk


In [ ]:
debug = True

In [ ]:
class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.grid()
        self.create_widgets()

    def create_widgets(self):
        self.green_label = tk.Label(self, text="Green CSV File:")
        self.green_label.grid(row=0, column=0)
        self.green_entry = tk.Entry(self)
        self.green_entry.grid(row=0, column=1)
        self.green_button = tk.Button(self, text="Browse", command=self.load_green_file)
        self.green_button.grid(row=0, column=2)

        self.red_label = tk.Label(self, text="Red CSV File:")
        self.red_label.grid(row=1, column=0)
        self.red_entry = tk.Entry(self)
        self.red_entry.grid(row=1, column=1)
        self.red_button = tk.Button(self, text="Browse", command=self.load_red_file)
        self.red_button.grid(row=1, column=2)

        self.paste_label = tk.Label(self, text="Paste Excel File:")
        self.paste_label.grid(row=2, column=0)
        self.paste_entry = tk.Entry(self)
        self.paste_entry.grid(row=2, column=1)
        self.paste_button = tk.Button(self, text="Browse", command=self.load_paste_file)
        self.paste_button.grid(row=2, column=2)

        self.run_button = tk.Button(self, text="Run", command=self.run_script)
        self.run_button.grid(row=3, column=0)
        self.exit_button = tk.Button(self, text="Exit", command=self.exit_application)
        self.exit_button.grid(row=3, column=1)

    def load_green_file(self):
        filename = filedialog.askopenfilename(filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
        if filename:
            self.green_entry.delete(0, tk.END)
            self.green_entry.insert(0, filename)
        
    def load_red_file(self):
        filename = filedialog.askopenfilename(filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))
        if filename:
            self.red_entry.delete(0, tk.END)
            self.red_entry.insert(0, filename)
    
    def load_paste_file(self):
        filename = filedialog.askopenfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")))
        if filename:
            self.paste_entry.delete(0, tk.END)
            self.paste_entry.insert(0, filename)

    def run_script(self):
        green_file = self.green_entry.get()
        red_file = self.red_entry.get()
        paste_file = self.paste_entry.get()

        if not green_file or not red_file or not paste_file:
            messagebox.showerror("Error", "Please select all input files!")
            return

        # Load the green and red CSV files
        df_green = pd.read_csv(green_file)
        df_red = pd.read_csv(red_file)

        # Load the paste Excel file
        wb = load_workbook(paste_file)
        paste_sheet = wb[wb.sheetnames[0]]

        # Check if the 'PCred' sheet exists and get the first row values
        if 'PCred' in wb.sheetnames:
            paste_sheet = wb['PCred']
            first_row_values = [cell.value for cell in paste_sheet[1]]

            # Create a new workbook for filtered data
            filtered_wb = Workbook()
            filtered_sheet = filtered_wb.active
            filtered_sheet.title = 'filtered data'

            for row_idx, row in enumerate(paste_sheet.iter_rows(min_row=1, min_col=1), start=1):
                for col_idx, cell in enumerate(row, start=1):
                    filtered_sheet.cell(row=row_idx, column=col_idx, value=cell.value)

            # Paste column 3 from green.csv into column 3 of filtered_data.xlsx, starting from row 2
            for index, value in enumerate(df_green['Mean'], start=2):
                filtered_sheet.cell(row=index, column=3, value=value)

            # Paste column 2 from red.csv into column 2 of filtered_data.xlsx, starting from row 2
            for index, value in enumerate(df_red['Mean'], start=2):
                filtered_sheet.cell(row=index, column=2, value=value)

            # Open a file dialog to choose the save location for filtered_data.xlsx
            save_path = filedialog.asksaveasfilename(defaultextension='.xlsx', filetypes=[('Excel files', '*.xlsx')],
                                                    initialfile='filtered_data.xlsx')

            if not save_path:
                return

            # Save the filtered data to the chosen file location
            filtered_wb.save(save_path)
            filtered_wb.close()

            # Close the paste.xlsx file
            wb.close()

            messagebox.showinfo("Success", "Script executed successfully!")

    def exit_application(self):
        self.master.destroy()

root = tk.Tk()
app = Application(master=root)
app.mainloop()

# Debug statements
if debug:
    if app.green_entry.get():
        print("✅ Green file was picked.")
    if app.red_entry.get():
        print("✅ Red file was picked.")
    if app.paste_entry.get():
        print("✅ Paste file was picked.")

    if 'filtered_wb' in locals() and 'paste_sheet' in locals():
        first_row_values = [cell.value for cell in paste_sheet[1]]
        for col_idx, value in enumerate(first_row_values, start=1):
            if filtered_sheet.cell(row=1, column=col_idx).value != value:
                print("❌ 'Copy the first row' not inputted correctly into filtered_data.xlsx.")
                break

        for row_idx, value in enumerate(paste_sheet.iter_rows(min_row=2, max_row=paste_sheet.max_row, min_col=1, max_col=1), start=2):
            if filtered_sheet.cell(row=row_idx, column=1).value != value[0].value:
                print("❌ 'Copy the first column' not inputted correctly into filtered_data.xlsx.")
                break

        for row in paste_sheet.iter_rows(min_row=2, max_row=paste_sheet.max_row, min_col=1, max_col=paste_sheet.max_column):
            for col_idx, cell in enumerate(row, start=1):
                if cell.data_type == 'f' and filtered_sheet.cell(row=cell.row, column=col_idx).value != cell.value:
                    print("❌ 'Copy formulas' not inputted correctly into filtered_data.xlsx.")
                    break

    if 'filtered_wb' in locals() and 'df_green' in locals() and 'df_red' in locals():
        for index, value in enumerate(df_green['Mean'], start=2):
            if filtered_sheet.cell(row=index, column=3).value != value:
                print("❌ 'Paste column 3 from green.csv' not inputted correctly into filtered_data.xlsx.")
                break

        for index, value in enumerate(df_red['Mean'], start=2):
            if filtered_sheet.cell(row=index, column=2).value != value:
                print("❌ 'Paste column 2 from red.csv' not inputted correctly into filtered_data.xlsx.")
                break

    if 'filtered_wb' in locals() and save_path:
        print("✅ 'filtered_data.xlsx' saved successfully.")
